In [1]:
# !pip3 -q install utils==1.0.1
# !pip3 -q install lightgbm==3.3.5

In [2]:
import warnings
# warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc,os,random
import time,datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold,GroupKFold

from utils import *
# root = args.root
# seed = args.seed
# remark = args.remark
# save_dir = args.save_dir
import lightgbm as lgb

## Config

In [3]:
root=''
remark=''
save_dir=''
seed=42
id_name = 'customer_ID'
label_name = 'target'

eps = 1e-3

## Processing

In [4]:
train = pd.read_parquet(f'train_data')
test = pd.read_parquet(f'test_data')

In [5]:
train_y =  pd.read_csv(f'{root}train_labels.csv')
train = train.merge(train_y, how='left',on=id_name)

print(train.shape,test.shape)

(5531451, 191) (11363762, 190)


In [9]:
pd.read_csv(f'{root}train_labels.csv').head()

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0


In [8]:
train.head(3)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,93.0,0.0,0.0,100.0,0.0,12.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,93.0,0.0,0.0,100.0,0.0,12.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,95.0,9.0,2.0,100.0,0.0,12.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0


## Run

In [11]:
def Metric(labels,preds):
    return amex_metric_mod(labels,preds)

def amex_metric_mod(y_true, y_pred):
        # y_true => 
        # 0          0
        # 1          0
        # 2          0
        #           ..
        # 5531449    0
        # 5531450    0
        # Name: target, Length: 2765213, dtype: int64
        
        # y_pred => [0.0018315  0.00164183 0.00174071 ... 0.00212767 0.00309472]
    labels     = np.transpose(np.array([y_true, y_pred]))    

    labels     = labels[labels[:, 1].argsort()[::-1]]
        # .argsort() => Return the integer indices that would sort the Series values.
        # .argsort()[::-1] => reverse the sorted indices.
        # labels => 
        # [[1.00000000e+00 9.97522107e-01]
        # [1.00000000e+00 9.97491614e-01]
        # ...
        # [0.00000000e+00 8.86596095e-05]
        # [0.00000000e+00 8.82186859e-05]]
        # labels[:,0] denotes originl targets, labels[:,1] denotes predicted targets.
        
    weights    = np.where(labels[:,0]==0, 20, 1)
        # weights => [ 1  1  1 ... 20 20 20]
        # np.cumsum(weights) => [       1        2        3 ... 42218046 42218066 42218086]
        # np.sum(weights) => 42218086
    
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
        # cut_vals =>     
        # [[1.         0.99752211]
        #  [1.         0.99749161]
        #  ...
        #  [1.         0.69189778]
        #  [0.         0.69189679]]    
    
        # np.sum(cut_vals[:,0]), np.sum(labels[:,0]) => 388343.0, 688746.0   
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
        # top_four => 0.5638406611435856

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]] # i = 1,0
            # when i==0 then labels => 
            # [[1.         0.67152673]
            #  [1.         0.6808289 ]
            #  ...
            #  [0.         0.5255297 ]
            #  [0.         0.0018315 ]]            
        weights        = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weights / np.sum(weights))
            # weight_random =>
            # [2.36865309e-08 4.73730619e-08 7.10595928e-08 ... 9.99999052e-01
            #  9.99999526e-01 1.00000000e+00]        
        total_pos      = np.sum(labels[:, 0] *  weights)
            # total_pos => 688746.0
        cum_pos_found  = np.cumsum(labels[:, 0] * weights)
            # cum_pos_found => 
            # [1.00000e+00 2.00000e+00 3.00000e+00 ... 6.88746e+05 6.88746e+05
            #  6.88746e+05]        
        lorentz        = cum_pos_found / total_pos
            # lorentz =>        
            # [1.45191406e-06 2.90382812e-06 4.35574217e-06 ... 1.00000000e+00
            #  1.00000000e+00 1.00000000e+00]        
        gini[i]        = np.sum((lorentz - weight_random) * weights)

    return 0.5 * (gini[1]/gini[0] + top_four)

def Write_log(logFile,text,isPrint=False):
    if isPrint:
        print(text)
    logFile.write(text)
    logFile.write('\n')
    return None

In [12]:
def Lgb_train_and_predict(train, test, config, gkf=False, aug=None, output_root='./output/', run_id=None):
    if not run_id:
        run_id = 'run_lgb_' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        while os.path.exists(output_root+run_id+'/'):
            time.sleep(1)
            run_id = 'run_lgb_' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        output_path = output_root + f'{save_dir}/'
    else:
        output_path = output_root + run_id + '/'
    if not os.path.exists(output_path):
        os.mkdir(output_path)
        
#     os.system(f'cp ./*.py {output_path}')
#     os.system(f'cp ./*.sh {output_path}')
    
    config['lgb_params']['seed'] = config['seed'] # config['seed'] = seed; defined in config.
    oof, sub = None, None
    if train is not None:
        log = open(output_path + '/train.log','w',buffering=1)
        log.write(str(config)+'\n')
        features = config['feature_name']
        params = config['lgb_params']
        rounds = config['rounds']
        verbose = config['verbose_eval']
        early_stopping_rounds = config['early_stopping_rounds']
        folds = config['folds'] # 5
        seed = config['seed']
        
        # assigning dataframe instead of series.
        oof = train[[id_name]] # id-name = 'customer_ID'
        oof[label_name] = 0 # creating column of zeros.

        all_valid_metric,feature_importance = [],[]
        if gkf: # group k fold. each customer_id appears once in one fold.
            # dropping duplicate customer id from set of ['customer_ID',target].
            tmp = train.loc[:,[id_name,label_name]].drop_duplicates(id_name).reset_index(drop=True)
                # tmp.columns => Index(['customer_ID', 'target'], dtype='object').
            skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
            split = skf.split(tmp,tmp[label_name])
            new_split = []
            for trn_index, val_index in split:
                # for first split => trn_index, val_index =>
                # [     1      2      3 ... 458905 458908 458911] [     0      4      5 ... 458909 458910 458912]
                
                # slice tmp, with row indexes present in 'trn_index', with column 'id_name'.
                trn_uids = tmp.loc[trn_index,id_name].values
                val_uids = tmp.loc[val_index,id_name].values
                new_split.append((train.loc[train[id_name].isin(trn_uids)].index,train.loc[train[id_name].isin(val_uids)].index))
            split = new_split
            del new_split
            _ = gc.collect()
            
            # skf = GroupKFold(n_splits=folds)
            # split = skf.split(train,train[label_name],train[id_name])
        else:
            skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
            split = skf.split(train,train[label_name])
            
        for fold, (trn_index, val_index) in enumerate(split):
            evals_result_dic = {}
            
            # training customer_ID's.
            train_cids = train.loc[trn_index,id_name].values
            
            if aug: # if augmentation.
                train_aug = aug.loc[aug[id_name].isin(train_cids)]
                trn_data = lgb.Dataset(train.loc[trn_index,features].append(train_aug[features]), label=train.loc[trn_index,label_name].append(train_aug[label_name]))
            else:
                trn_data = lgb.Dataset(train.loc[trn_index,features], label=train.loc[trn_index,label_name])

            val_data = lgb.Dataset(train.loc[val_index,features], label=train.loc[val_index,label_name])
            model = lgb.train(params,
#                 init_model = output_path + '/fold0.ckpt', # continue training from previous save.
                init_model = output_path + '/fold%s.ckpt'%fold, 
                train_set  = trn_data,
                num_boost_round   = rounds,
                valid_sets = [trn_data,val_data],
                evals_result = evals_result_dic,
                early_stopping_rounds = early_stopping_rounds,
                verbose_eval = verbose
            )

#             model = lgb.Booster(model_file=output_path + '/fold%s.ckpt'%fold) # loading model from saved ckpt.
            model.save_model(output_path + '/fold%s.ckpt'%fold)

            valid_preds = model.predict(train.loc[val_index,features], num_iteration=model.best_iteration)
                # valid_preds => [0.0018315  0.00164183 0.00174071 ... 0.00257369 0.00212767 0.00309472]

            # filling a slice of dataframe.
            # replacing actual targets with predictions.
            oof.loc[val_index,label_name] = valid_preds

#             for i in range(len(evals_result_dic['valid_1'][params['metric']])//verbose):
#                 Write_log(log,' - %i round - train_metric: %.6f - valid_metric: %.6f\n'%(i*verbose,evals_result_dic['training'][params['metric']][i*verbose],evals_result_dic['valid_1'][params['metric']][i*verbose]))

            all_valid_metric.append(Metric(train.loc[val_index,label_name],valid_preds))
            Write_log(log,'- fold%s valid metric: %.6f\n'%(fold,all_valid_metric[-1]))
            
            # Get feature importances. importance_type; How the importance is calculated. 
            # importance_type=“split” => result contains numbers of times the feature is used in a model.
            # importance_type=“gain” => result contains total gains of splits which use the feature.
            importance_gain = model.feature_importance(importance_type='gain')            
            importance_split = model.feature_importance(importance_type='split')
            
            feature_name = model.feature_name()
            feature_importance.append(pd.DataFrame({'feature_name':feature_name,'importance_gain':importance_gain,'importance_split':importance_split}))            
        
        # Concatenating pandas along axis=0.
        feature_importance_df = pd.concat(feature_importance)
        
        feature_importance_df = feature_importance_df.groupby(['feature_name']).mean().reset_index()
        feature_importance_df = feature_importance_df.sort_values(by=['importance_gain'],ascending=False)
        feature_importance_df.to_csv(output_path + '/feature_importance.csv',index=False)

        mean_valid_metric = np.mean(all_valid_metric)
        global_valid_metric = Metric(train[label_name].values,oof[label_name].values)
        Write_log(log,'all valid mean metric:%.6f, global valid metric:%.6f'%(mean_valid_metric,global_valid_metric))

        oof.to_csv(output_path + '/oof.csv',index=False)

        log.close()
        os.rename(output_path + '/train.log', output_path + '/train_%.6f.log'%mean_valid_metric)

        log_df = pd.DataFrame({'run_id':[run_id],'mean metric':[round(mean_valid_metric,6)],'global metric':[round(global_valid_metric,6)],'remark':[remark]})
        if not os.path.exists(output_root + '/experiment_log.csv'):
            log_df.to_csv(output_root + '/experiment_log.csv',index=False)
        else:
            log_df.to_csv(output_root + '/experiment_log.csv',index=False,header=None,mode='a')
            # mode='a' => open for writing, appending to the end of file if it exists.

    if test is not None:
        sub = test[[id_name]] # assigning dataframe instead of series.
        sub['prediction'] = 0
        for fold in range(folds):
            model = lgb.Booster(model_file=output_path + '/fold%s.ckpt'%fold)
            test_preds = model.predict(test[features], num_iteration=model.best_iteration)
            sub['prediction'] += (test_preds / folds)
        sub[[id_name,'prediction']].to_csv(output_path + '/submission.csv.zip', compression='zip',index=False)
#     if save_dir in output_path:
#         os.rename(output_path,output_root+run_id+'/')
        
#     return oof,sub,(mean_valid_metric,global_valid_metric)

In [13]:
lgb_config = {
    'lgb_params':{
        'objective' : 'binary', # binary log loss classification (or logistic regression).
        'metric' : 'binary_logloss',
        'boosting': 'dart', # Dropouts meet Multiple Additive Regression Trees.
        'max_depth' : -1, # limit the max depth for tree model. <= 0 means no limit.
        'num_leaves' : 64, # max number of leaves in one tree.
        'learning_rate' : 0.015, # 0.035
        'bagging_freq': 5, # frequency for bagging, bagging at every kth (5th) iteration.
                            #Every k-th (5-th) iteration, LightGBM will randomly select bagging_fraction * 100 % of the data to use for the next k iterations
        
        'bagging_fraction' : 0.7,
        'feature_fraction' : 0.7, # LightGBM will select 70% of features before training each tree (iteration).
        'min_data_in_leaf': 256, # minimal number of data in one leaf.
        'max_bin': 63, # max number of bins that feature values will be bucketed in.
        'min_data_in_bin': 256, # minimal number of data inside 'one bin'.
        # 'min_sum_heassian_in_leaf': 10,
        'tree_learner': 'serial', # single machine tree learner.
        'boost_from_average': 'false', # adjusts initial score to the mean of labels for faster convergence.
        'lambda_l1' : 0.1, # L1 regularization.
        'lambda_l2' : 30, # L2 regularization
        'num_threads': 11, # for the best speed, set this to the number of real CPU cores.
        'verbosity' : 1,
#         'device' : 'gpu'
    },
    'feature_name':[col for col in train.columns if col not in [id_name,label_name,'S_2']],
    'rounds':50, # 4500
    'early_stopping_rounds':100,
    'verbose_eval':50,
    'folds':5, # 5, (1; for deubg)
    'seed':seed
}

Lgb_train_and_predict(train,test,lgb_config,gkf=True,aug=None, output_root='./o_debug/', run_id='LGB_with_series_feature')

/home/na/miniconda3/envs/base_2/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/na/miniconda3/envs/base_2/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/na/miniconda3/envs/base_2/lib/python3.7/site-packages/ipykernel_launcher.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

(                                               customer_ID    target
 0        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.001716
 1        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.001540
 2        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.001630
 3        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.001637
 4        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.001119
 ...                                                    ...       ...
 5531446  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  0.003640
 5531447  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  0.002444
 5531448  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  0.002510
 5531449  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  0.002056
 5531450  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  0.003002
 
 [5531451 rows x 2 columns],
                                                 customer_ID  prediction
 0         00000469ba478561f23a92a868bd366de6f6527a684c9

In [16]:
pd.read_csv('o_debug/experiment_log.csv')
# mean_metric = mean of folds.
# global_metric = whole dataframe at once.

,run_id,mean metric,global metric,remark
0,LGB_with_series_feature,0.731081,0.730968,NaN
1,LGB_with_series_feature,0.731057,0.730957,NaN
2,LGB_with_series_feature,0.731057,0.730957,NaN


In [14]:
pd.read_csv('o_debug/LGB_with_series_feature/oof.csv').head(3)

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.001716
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.001540
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.001630


In [15]:
pd.read_csv('o_debug/LGB_with_series_feature/feature_importance.csv').head(3)

,feature_name,importance_gain,importance_split
0,P_2,1.373261e+08,11535.0
1,B_9,1.901178e+07,2985.2
2,D_48,1.670780e+07,3578.8
